import libraries

In [1]:
from fastai.vision import *
from fastbook import *

matplotlib.rc('image', cmap='Greys')



Grab Dataset

In [2]:
mnist_path = untar_data(URLs.MNIST)

In [3]:
# re-visit
Path.BASE_PATH = mnist_path

In [4]:
mnist_path.ls()

(#2) [Path('training'),Path('testing')]

In [5]:
(mnist_path/'training').ls()

(#10) [Path('training/4'),Path('training/7'),Path('training/9'),Path('training/5'),Path('training/8'),Path('training/0'),Path('training/2'),Path('training/1'),Path('training/6'),Path('training/3')]

Assign Sets from path

In [7]:
nines = get_files(mnist_path/'training'/'9')
sixes = get_files(mnist_path/'training'/'6')

Create Tensors from each image in path

Stack the list of tensors

Get the average of the stacks and assign

Assign arbitrary actual image

Assign to calculate distance from example to ideal
1. The mean absolute distance(L1 Norm)
1. The squareroot of the mean of the square of differences(root mean squared error, or L2 Norm)

Do the same as above with the pytorch library

*** 

Create tensors for the validation set of each

Define a function that returns mean absolute distance(L1 Norm)(-1,-2)

Pass sample image and mean image to function

Define a function that returns a boolean `True` value if a sample image is a 3

Assign the return value of the above as the mean of a float to calculate accuracy of 3's and 7's

*** 

Create a simple quadratic function definition

Plot the function

Scatter Plot the function

*** 

Create a scalar tensor `xt` (float) and "tag" that we want to calculate gradients with respect to that variable

Calucate our function `yt` at funtion `xt`

Tell pytorch to calculate the gradients of `yt` for us

View the gradients that were calculated

*** 

Repeat the above using the same name(`xt`), but create a vector

Add a `sum` to the above function, so that it can take a vector and return a scalar

Repeat backpropagation

*** 

*Imagine you were measuring the speed of a roller coaster as it went over the top of a hump. It would start fast, and then get slower as it went up the hill; it would be slowest at the top, and it would then speed up again as it went downhill. You want to build a model of how the speed changes over time*

Create a vector tensor `time` as 20 float values

Create at vector tensor `speed` at 20 float values that would show a random quadratic distribution of points

Plot result

Define a quadratic function(`f`) that takes 2 arguments
- one for time
- one for parameters

In this function separate the input and the parameters in the return value

Define a function that calculates the means square error on predictions and targets

#### Step 1: Initialize the parameters

Also clone original params for graphs later

#### Step 2: Calculate the predictions

#### Step 3: Calculate the loss

#### Step 4: Calculate the gradients

Pick a learning rate

#### Step 5: Step the weights. 

Update parameters based on calculated gradients

Check if loss has improved

Create a function so that all the above can be applied several times

#### Step 6: Repeat the process 

Iterate to repeat the process

#### Step 7: stop

*** 

Create a graphvis of the gradient descent process

*** 